# Preparação dos Dados do Ibovespa

Este notebook realiza a preparação dos dados históricos do Ibovespa, incluindo a limpeza e a criação de novas features para a análise subsequente.

## Passo 1: Importar Bibliotecas Necessárias

Primeiro, vamos importar as bibliotecas que utilizaremos no processo de preparação dos dados.


In [36]:
import pandas as pd
import numpy as np

## Passo 2: Carregar os Dados

Vamos carregar o arquivo CSV contendo os dados históricos do Ibovespa.

Aqui, carregamos os dados históricos do Ibovespa a partir de um arquivo CSV. Inspecionamos as primeiras linhas para garantir que os dados foram carregados corretamente.


In [37]:
# Carregar os dados
data_path = 'dados-historicos-ibovespa.csv'
df = pd.read_csv(data_path)
df.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,12.07.2024,128.897,128.293,129.015,128.002,"7,21B","0,47%"
1,11.07.2024,128.294,127.221,128.326,127.221,"7,97B","0,85%"
2,10.07.2024,127.218,127.109,127.769,126.928,"8,15B","0,09%"
3,09.07.2024,127.108,126.548,127.295,125.937,"6,68B","0,44%"
4,08.07.2024,126.548,126.280,126.551,125.614,"7,43B","0,22%"


## Passo 3: Converter a Coluna 'Data' para datetime

Precisamos garantir que a coluna `Data` está no formato datetime para facilitar a manipulação de datas.

Neste passo, convertemos a coluna `Data` para o formato datetime para garantir que as datas sejam tratadas corretamente durante a análise.


In [38]:
# Converter a coluna 'Data' para datetime
df['Data'] = pd.to_datetime(df['Data'], format='%d.%m.%Y')

# Função para converter colunas numéricas
def convert_column(column):
    if df[column].dtype == 'object':
        df[column] = df[column].str.replace('%', '').str.replace(',', '.')
    df[column] = pd.to_numeric(df[column])

# Converter colunas específicas
convert_column('Último')
convert_column('Abertura')
convert_column('Máxima')
convert_column('Mínima')
convert_column('Var%')

df.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,2024-07-12,128.897,128.293,129.015,128.002,"7,21B",0.47
1,2024-07-11,128.294,127.221,128.326,127.221,"7,97B",0.85
2,2024-07-10,127.218,127.109,127.769,126.928,"8,15B",0.09
3,2024-07-09,127.108,126.548,127.295,125.937,"6,68B",0.44
4,2024-07-08,126.548,126.280,126.551,125.614,"7,43B",0.22


## Passo 4: Converter a Coluna 'Vol.' para um Formato Numérico

A coluna 'Vol.' contém valores representados em diferentes unidades (K, M, B). Vamos criar uma função para converter esses valores para um formato numérico consistente.

In [39]:
# Função para converter os valores da coluna 'Vol.'
def convert_volume_corrected(vol):
    if isinstance(vol, str):
        if 'B' in vol:
            return float(vol.replace('B', '').replace(',', '.')) * 1e9  # Bilhões
        elif 'M' in vol:
            return float(vol.replace('M', '').replace(',', '.')) * 1e6  # Milhões
        elif 'K' in vol:
            return float(vol.replace('K', '').replace(',', '.')) * 1e3  # Milhares
        else:
            return float(vol.replace(',', '.'))
    return vol

# Aplicar a função na coluna 'Vol.'
df['Vol.'] = df['Vol.'].apply(convert_volume_corrected)
df.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,2024-07-12,128.897,128.293,129.015,128.002,7.210000e+09,0.47
1,2024-07-11,128.294,127.221,128.326,127.221,7.970000e+09,0.85
2,2024-07-10,127.218,127.109,127.769,126.928,8.150000e+09,0.09
3,2024-07-09,127.108,126.548,127.295,125.937,6.680000e+09,0.44
4,2024-07-08,126.548,126.280,126.551,125.614,7.430000e+09,0.22


## Passo 5: Remover Linhas com Valores Ausentes

Vamos remover qualquer linha que contenha valores ausentes para garantir a integridade dos dados.


In [40]:
# Remover linhas com valores ausentes
df.dropna(inplace=True)
df.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,2024-07-12,128.897,128.293,129.015,128.002,7.210000e+09,0.47
1,2024-07-11,128.294,127.221,128.326,127.221,7.970000e+09,0.85
2,2024-07-10,127.218,127.109,127.769,126.928,8.150000e+09,0.09
3,2024-07-09,127.108,126.548,127.295,125.937,6.680000e+09,0.44
4,2024-07-08,126.548,126.280,126.551,125.614,7.430000e+09,0.22


## Passo 6: Adicionar Marcadores de Eventos Econômicos

Vamos adicionar marcadores de eventos econômicos aos dados. Esses eventos podem ter impacto significativo no mercado e são importantes para nossas análises.


In [41]:
# Função para adicionar eventos aos dados
def add_event_markers(df, events):
    df['Evento'] = 'Nenhum'
    for event, dates in events.items():
        start_date, end_date = dates
        df.loc[(df['Data'] >= start_date) & (df['Data'] <= end_date), 'Evento'] = event
    return df

# Definir os eventos econômicos e suas datas
eventos = {
    'Recessão Econômica Brasileira': ('2014-01-01', '2016-12-31'),
    'Impeachment de Dilma Rousseff': ('2016-04-17', '2016-08-31'),
    'Eleição Presidencial 2018': ('2018-10-07', '2018-10-28'),
    'Pandemia de COVID-19': ('2020-02-26', '2020-12-31'),
    'Eleição Presidencial 2022': ('2022-10-02', '2022-10-30'),
    'Recuperação Pós-Pandemia': ('2023-01-01', '2024-07-12')
}

# Adicionar os marcadores de eventos aos dados
df = add_event_markers(df, eventos)
df.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Evento
0,2024-07-12,128.897,128.293,129.015,128.002,7.210000e+09,0.47,Recuperação Pós-Pandemia
1,2024-07-11,128.294,127.221,128.326,127.221,7.970000e+09,0.85,Recuperação Pós-Pandemia
2,2024-07-10,127.218,127.109,127.769,126.928,8.150000e+09,0.09,Recuperação Pós-Pandemia
3,2024-07-09,127.108,126.548,127.295,125.937,6.680000e+09,0.44,Recuperação Pós-Pandemia
4,2024-07-08,126.548,126.280,126.551,125.614,7.430000e+09,0.22,Recuperação Pós-Pandemia


## Passo 7: Criar Variáveis Dummy para os Eventos Econômicos

Vamos transformar a coluna de eventos em variáveis dummy para facilitar a análise e modelagem.

In [42]:
# Criar variáveis dummy para os eventos econômicos
df = pd.get_dummies(df, columns=['Evento'], drop_first=True)
df.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Evento_Eleição Presidencial 2022,Evento_Impeachment de Dilma Rousseff,Evento_Nenhum,Evento_Pandemia de COVID-19,Evento_Recessão Econômica Brasileira,Evento_Recuperação Pós-Pandemia
0,2024-07-12,128.897,128.293,129.015,128.002,7.210000e+09,0.47,False,False,False,False,False,True
1,2024-07-11,128.294,127.221,128.326,127.221,7.970000e+09,0.85,False,False,False,False,False,True
2,2024-07-10,127.218,127.109,127.769,126.928,8.150000e+09,0.09,False,False,False,False,False,True
3,2024-07-09,127.108,126.548,127.295,125.937,6.680000e+09,0.44,False,False,False,False,False,True
4,2024-07-08,126.548,126.280,126.551,125.614,7.430000e+09,0.22,False,False,False,False,False,True


## Passo 8: Aplicar Transformação Logarítmica na Coluna de Volume

Para reduzir a variabilidade dos dados de volume, vamos aplicar uma transformação logarítmica.

In [43]:
# Aplicar transformação logarítmica na coluna de volume
df['Vol_log'] = np.log1p(df['Vol.'])
df.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Evento_Eleição Presidencial 2022,Evento_Impeachment de Dilma Rousseff,Evento_Nenhum,Evento_Pandemia de COVID-19,Evento_Recessão Econômica Brasileira,Evento_Recuperação Pós-Pandemia,Vol_log
0,2024-07-12,128.897,128.293,129.015,128.002,7.210000e+09,0.47,False,False,False,False,False,True,22.698735
1,2024-07-11,128.294,127.221,128.326,127.221,7.970000e+09,0.85,False,False,False,False,False,True,22.798950
2,2024-07-10,127.218,127.109,127.769,126.928,8.150000e+09,0.09,False,False,False,False,False,True,22.821284
3,2024-07-09,127.108,126.548,127.295,125.937,6.680000e+09,0.44,False,False,False,False,False,True,22.622384
4,2024-07-08,126.548,126.280,126.551,125.614,7.430000e+09,0.22,False,False,False,False,False,True,22.728792


## Passo 9: Salvar os Dados Preparados

Finalmente, vamos salvar os dados preparados em um novo arquivo CSV para serem usados nas próximas etapas da análise.


In [44]:
# Salvar os dados preparados
df.to_csv('dados_preparados.csv', index=False)